# Assignemt 1

In [ ]:
import cv2
import numpy as np
from dlvc.datasets.pets import PetsDataset, Subset
from assignments.src.dlvc import ops
from assignments.src.dlvc.batches import BatchGenerator
from assignments.src.dlvc.dataset import Dataset, ClassificationDataset

fdir = '/mnt/1028D91228D8F7A4/Python Project/PycharmProjects/DeepLearning/assignments/src/cifar10'
pds = PetsDataset(fdir, Subset.TRAINING)
print('Xshape, yshape, len, n class: ', pds.X.shape, pds.y.shape, len(pds), pds.num_classes())
print('dtype X,y: ', pds.X.dtype, pds.y.dtype)
print('sample: ', pds[0])
print('y. ', pds.y[0:10])

cv2.imwrite('sample.png', pds[10][1])

In [ ]:

#test part 2 assignment 1
op = ops.chain([
    ops.vectorize(),
    ops.type_cast(np.float32),
    ops.add(-127.5),
    ops.mul(1/127.5),
])

In [ ]:
train_ds = PetsDataset(fdir, Subset.TRAINING)
bg = BatchGenerator(train_ds, 10, False, op)

In [ ]:
from dlvc.datasets.pets import PetsDataset
from dlvc.dataset import Subset
from dlvc.batches import BatchGenerator
from dlvc.test import Accuracy
import dlvc.ops as ops
import numpy as np
import torch


# TODO: Define the network architecture of your linear classifier.
class LinearClassifier(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LinearClassifier, self).__init__()

        self.input_dim = input_dim
        self.num_classes = num_classes

        # define network layer
        self.layer = torch.nn.Linear(self.input_dim, self.num_classes)

    def forward(self, x):
        return self.layer(x)


op = ops.chain([
    ops.vectorize(),
    ops.type_cast(np.float32),
    ops.add(-127.5),
    ops.mul(1 / 127.5),
])


def train_model(linear_classifier, criterion, optimizer, epochs, train_data, valid_data):
    acc = Accuracy()
    print("Train the network")
    accs = np.zeros(epochs)
    best_acc = 0.0
    for epoch in range(epochs):
        running_loss = 0.0
        acc.reset()
        for data in train_data:
            # get the inputs and the labels
            inputs = data.data
            labels = data.label

            # convert the np.array in tensor
            t_inputs = torch.tensor(inputs)
            t_labels = torch.tensor(labels).to(torch.long) #cast labels to long so the CE works (CE throws exception with int)

            # zero the parameter gradients, for every batch I must compute the gradient again
            optimizer.zero_grad()

            # forward step
            output = linear_classifier.forward(t_inputs)
            loss = criterion(output, t_labels)
            loss.backward()  # compute the gradients
            optimizer.step()  # update the parameter

            # print statistics
            running_loss += loss.item()
            acc.update(linear_classifier.forward(torch.tensor(valid_data.data)).detach().numpy(), valid_data.label)
            print(f"epoch {epoch + 1} \ntrain loss: {running_loss}\nval accuracy: {acc.accuracy()}")
            accs[epoch] = acc.accuracy()
            
            # update the values
            running_loss = 0.0
            if acc.accuracy() >= best_acc:
                best_acc = acc.accuracy()

            acc.reset()
    print("Finished Training")
    return linear_classifier, best_acc, accs


def main():
    fp = 'C:/Users/admin/Desktop/10. Semester/Computer Vision/dlvc_ss22/assignments/reference/cifar10'

    print("Load data")
    train_ds = PetsDataset(fp, Subset.TRAINING)
    valid_ds = PetsDataset(fp, Subset.VALIDATION)
    test_ds = PetsDataset(fp, Subset.TEST)
    print("Data Loaded")

    print("Creating Batch Generator")
    train = BatchGenerator(train_ds, len(train_ds), False, op)
    valid = next(iter(BatchGenerator(valid_ds, len(valid_ds), False, op)))
    test = next(iter(BatchGenerator(test_ds, len(test_ds), False, op)))
    print("Batch Generator created")

    #define general parameters
    in_features = 3072  # size of the vector in input
    epochs = 100

    lc = LinearClassifier(in_features, train_ds.num_classes())
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(lc.parameters(), lr=0.001)
    lc, best_val_acc, val_accs = train_model(lc, criterion, optimizer, epochs, train, valid)


    # compute the test accuracy
    test_acc = Accuracy()
    test_acc.update(lc.forward(torch.tensor(test.data)).detach().numpy(), test.label)
    print(f"test accuracy: {test_acc.accuracy()}")
    
    return val_accs, best_val_acc, test_acc


val_accs, best_val_acc, test_acc = main()
print(val_accs, best_val_acc, test_acc)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(val_accs)
plt.xlabel('Validation Accuracy')
plt.ylabel('epochs')
plt.show()

# Assignment 2

In [ ]:

class ResBlock(nn.Module):
    def __init__(self, c_in: int, c_out: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(c_in, c_out, 3, padding='same')
        self.bn1 = nn.BatchNorm2d(c_out)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(c_out, c_in, 3, padding='same')
        self.bn2 = nn.BatchNorm2d(c_out)

    def forward(self, x: Tensor) -> Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, input_channels, n_classes) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, 3, padding='same')
        self.block1 = ResBlock(64,64)
        self.downsample1 = nn.Conv2d(64, 128, 3, stride=2)
        self.block2 = ResBlock(128,128)
        self.downsample2 = nn.Conv2d(128, 256, 3, stride=2)
        self.block3 = ResBlock(256,256)
        self.fc = nn.Linear(12544, n_classes) # TODO change 400 to correct
        
    def forward(self, x: Tensor) -> Tensor:
        out = self.conv1(x)
        out = self.block1(out)
        out = self.downsample1(out)
        out = self.block2(out)
        out = self.downsample2(out)
        out = self.block3(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


In [1]:
from dlvc.datasets.pets import PetsDataset
from dlvc.dataset import Subset
from dlvc.batches import BatchGenerator
from dlvc.test import Accuracy
import dlvc.ops as ops
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

from dlvc.models.pytorch import CnnClassifier

class CNN(nn.Module):
    def __init__(self, c_in) -> None:
        super(CNN, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(c_in, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(576, 120)  # 400 ergibt sich bei input image 28x28
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: Tensor) -> Tensor:
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def run_experiment(fp, net, name, n_epochs, batch_size, input_shape, num_classes, shuffle, lr, wd, op):
    train_ds = PetsDataset(fp, Subset.TRAINING)
    valid_ds = PetsDataset(fp, Subset.VALIDATION)
    test_ds = PetsDataset(fp, Subset.TEST)

    train = BatchGenerator(train_ds, batch_size, shuffle, op)
    valid = BatchGenerator(valid_ds, batch_size, shuffle, op)
    test = BatchGenerator(test_ds, batch_size, shuffle, op)

    model = CnnClassifier(net, input_shape, num_classes, lr, wd)
    acc = Accuracy()

    for epoch in range(n_epochs):
        losses = []
        best_acc = 0
        # Training Loop
        for data in train:
            loss = model.train(data.data, data.label)
            losses.append(loss)
        # Validation Loop
        acc.reset()
        for data in valid:
            pred = model.predict(data.data)
            acc.update(pred, data.label)

        # Reporting
        losses = np.array(losses)

        print('epoch', epoch)
        print('train loss:', losses.mean(), '±', losses.std())
        print('val acc:', acc.accuracy())

        if acc.accuracy() > best_acc:
            torch.save(model, f'best_{name}.pth')
            best_acc = acc.accuracy()

    model = torch.load(f'best_{name}.pth')
    acc.reset()
    for data in test:
        pred = model.predict(data.data)
        acc.update(pred, data.label)
    print('--------------------')
    print('val acc (best):', best_acc)
    print('test acc:', acc.accuracy())
    return best_acc, acc.accuracy()





op_no_da = ops.chain([
    ops.type_cast(np.float32),
    ops.add(-127.5),
    ops.mul(1 / 127.5),
    ops.hwc2chw(),
])

op_da = ops.chain([
    ops.type_cast(np.float32),
    ops.add(-127.5),
    ops.mul(1 / 127.5),
    ops.hflip(),
    ops.hwc2chw(),
])

# try neither, only wd weak, only wd medium, only wd overreg, only da, both
# a config contains [wd, ops]
configs = [
    [0.01, op_da, 'cnn_low_wd_and_hf']
]

# Constants
fp = 'C:/Users/admin/Desktop/10. Semester/Computer Vision/dlvc_ss22/assignments/reference/cifar10'
n_epochs = 100
batch_size=128
input_shape = (0, 3, 32, 32)
num_classes = 2
shuffle = True
lr = 0.01

net = CNN(input_shape[1])
# move to cuda if available
net.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

results = []
for wd, op, name in configs:
    best_val_acc, test_acc = run_experiment(fp, net, name, n_epochs, batch_size, input_shape, num_classes, shuffle, lr, wd, op)
    results.append([name, best_val_acc, test_acc])
    
print(results)

epoch 0
train loss: 1.103727174183679 ± 0.5451035468071038
val acc: 0.4977951984321411
epoch 1
train loss: 0.6950364037165566 ± 0.006131209468592341
val acc: 0.4977951984321411
epoch 2
train loss: 0.6905534797244601 ± 0.007104778060515808
val acc: 0.512983831455169
epoch 3
train loss: 0.6831457633820791 ± 0.011841955183205119
val acc: 0.5590396864282214
epoch 4
train loss: 0.6740419136153327 ± 0.01739311971511226
val acc: 0.5918667319941205
epoch 5
train loss: 0.6666214390406533 ± 0.01957517401615726
val acc: 0.6050955414012739
epoch 6
train loss: 0.6596668504533314 ± 0.02245154883965912
val acc: 0.622733953944145
epoch 7
train loss: 0.6561755850201562 ± 0.024370831389295047
val acc: 0.6266536011758942
epoch 8
train loss: 0.6527470208349682 ± 0.026235492758105716
val acc: 0.622733953944145
epoch 9
train loss: 0.6507221081900219 ± 0.026508966451239072
val acc: 0.6330230279274865
epoch 10
train loss: 0.648326125409868 ± 0.02730340429748494
val acc: 0.6340029397354238
epoch 11
train loss: